In [32]:
import os
import fitz  # PyMuPDF
import re
import openai

openai.api_type = "azure"
openai.api_base = "https://pce-aiservices600098751.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = "abe66c9c81b74e8d9a191f5a8b7932cc"

# Open the PDF file
pdf_file_path = "C:/Users/VedantSoni/Desktop/lol.pdf"

# Check if the file exists
if not os.path.exists(pdf_file_path):
    print(f"Error: File '{pdf_file_path}' not found.")
    exit()

try:
    # Read the PDF file using PyMuPDF
    pdf_file = fitz.open(pdf_file_path)

    # Accumulate text from all pages
    all_text = ""
    for page_num in range(len(pdf_file)):
        page = pdf_file.load_page(page_num)
        all_text += page.get_text().lower()
    
    # Make a single API call to generate the decision and reason
    response = openai.ChatCompletion.create(
        engine="GPT-4",
        messages=[
            {
                "role": "system",
                "content": "You are an expert systematic literature reviewer. This is Full Text Screening phase you have to analyze the entire document provided to you. Based on the below exclusion and inclusion criteria, please classify the given Citation as 'Included' or 'Excluded'. Apply the Exclusion Criteria first. If any statement in the Exclusion Criteria is true, mark the citation as 'Excluded'. If the citation passes the Exclusion Criteria, then check the Inclusion Criteria. Mark the Citation as 'Included' only if it strictly and exactly passes all Inclusion Criteria statements; otherwise, mark the citation as 'Excluded'. Decision: <your decision either Include or Exclude> and Reason: <why you took that decision>.",
            },
            {
                "role": "user",
                "content": f"""
                Inclusion Criteria:
                1. Study Design: Only randomized controlled trials (RCTs) are included. No other study design is acceptable.
                2. Condition: The study must explicitly mention 'post-traumatic stress disorder'.
                3. Medications: Include studies that primarily investigate the utilization of the following medications: sertraline, 'sertraline plus brexpiprazole', fluoxetine, paroxetine, and venlafaxine. These medications are permissible for use either as standalone treatments (monotherapy) or in combination therapy. However, in cases of combination therapy, it is imperative that the combination exclusively involves the aforementioned medications.
                4. Population: The study must involve adult patients aged 18 years and older.
                5. If the trial is assessing more than two interventions in different arms of the randomized control trial, at least two interventions should be from sertraline, 'sertraline plus brexpiprazole', fluoxetine, paroxetine, and venlafaxine
                Exclusion Criteria:
                1. Study Type: Literature reviews, meta-analyses, systematic reviews, review, narrative review, observational studies (including retrospective, case-control, cohort studies, pharmacokinetic, pharmacodynamics study, cost-effectiveness, economic evaluation studies, longitudinal studies, and cross-sectional studies).
                2. Population: Studies exclusively involving animals, adolescents, children, or populations other than those with post-traumatic stress disorder.
                3. Medications: Exclude studies that do not primarily investigate the utilization of the following medications: sertraline, 'sertraline plus brexpiprazole', fluoxetine, paroxetine, and venlafaxine. These medications are permissible for use either as standalone treatments (monotherapy) or in combination therapy. However, in cases of combination therapy, it is imperative that the combination exclusively involves the aforementioned medications.
                4. Strictly mark the citation as excluded if it mentions review or pooled analysis which includes data from more than one study.
                5. If study design is not clearly randomized controlled trials (RCTs) then exclude the study.
                """,
            },
            {
                "role": "user",
                "content": all_text,
            },
        ],
    )

    # Get the decision and reason
    decision = response["choices"][0]["message"]["content"]

    # Write the decision to a text file
    pdf_decision_file = pdf_file_path.replace(
        os.path.splitext(pdf_file_path)[1], "_decision.txt"
    )
    with open(pdf_decision_file, "w+") as file:
        file.write(decision)

    # Print the decision
    print(decision)

except Exception as e:
    print(f"An error occurred: {str(e)}")


An error occurred: This model's maximum context length is 8192 tokens. However, your messages resulted in 9502 tokens. Please reduce the length of the messages.


In [31]:
import os
import fitz  # PyMuPDF
import openai

openai.api_type = "azure"
openai.api_base = "https://pce-aiservices600098751.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = "abe66c9c81b74e8d9a191f5a8b7932cc"

# Open the PDF file
pdf_file_path = "C:/Users/VedantSoni/Desktop/test.pdf"

# Function to chunk text into smaller parts
def chunk_text(text, chunk_size=3800):
    chunks = []
    for i in range(0, len(text), chunk_size):
        chunks.append(text[i:i + chunk_size])
        print(chunks)
    return chunks
    
# Function to send chunk and receive decision
def send_chunk_get_decision(chunk, context=""):
    response = openai.ChatCompletion.create(
        engine="GPT-4",
        messages=[
            {
                "role": "system",
                "content": prompt,
            },
            {
                "role": "user",
                "content": chunk,
            },
        ],
        temperature=0.9,
        max_tokens=150,
        stop=["\n\n"]
    )
    return response["choices"][0]["message"]["content"]



# Check if the file exists
if not os.path.exists(pdf_file_path):
    print(f"Error: File '{pdf_file_path}' not found.")
    exit()

# Initialize decision to accumulate results
decision = ""

try:
    # Read the PDF file using PyMuPDF
    pdf_file = fitz.open(pdf_file_path)

    # Prompt for GPT-4
    prompt = """
    You are an expert systematic literature reviewer. This is the Full Text Screening phase. You have to analyze the entire document provided to you. Based on the below exclusion and inclusion criteria, please classify the given citation as 'Included' or 'Excluded'. Apply the Exclusion Criteria first. If any statement in the Exclusion Criteria is true, mark the citation as 'Excluded'. If the citation passes the Exclusion Criteria, then check the Inclusion Criteria. Mark the citation as 'Included' only if it strictly and exactly passes all Inclusion Criteria statements; otherwise, mark the citation as 'Excluded'. 

    Inclusion Criteria:
    1. Study Design: Only randomized controlled trials (RCTs) are included. No other study design is acceptable.
    2. Condition: The study must explicitly mention 'post-traumatic stress disorder'.
    3. Medications: Include studies that primarily investigate the utilization of the following medications: sertraline, 'sertraline plus brexpiprazole', fluoxetine, paroxetine, and venlafaxine. These medications are permissible for use either as standalone treatments (monotherapy) or in combination therapy. However, in cases of combination therapy, it is imperative that the combination exclusively involves the aforementioned medications.
    4. Population: The study must involve adult patients aged 18 years and older.
    5. If the trial is assessing more than two interventions in different arms of the randomized control trial, at least two interventions should be from sertraline, 'sertraline plus brexpiprazole', fluoxetine, paroxetine, and venlafaxine

    Exclusion Criteria:
    1. Study Type: Literature reviews, meta-analyses, systematic reviews, review, narrative review, observational studies (including retrospective, case-control, cohort studies, pharmacokinetic, pharmacodynamics study, cost-effectiveness, economic evaluation studies, longitudinal studies, and cross-sectional studies).
    2. Population: Studies exclusively involving animals, adolescents, children, or populations other than those with post-traumatic stress disorder.
    3. Medications: Exclude studies that do not primarily investigate the utilization of the following medications: sertraline, 'sertraline plus brexpiprazole', fluoxetine, paroxetine, and venlafaxine. These medications are permissible for use either as standalone treatments (monotherapy) or in combination therapy. However, in cases of combination therapy, it is imperative that the combination exclusively involves the aforementioned medications.
    4. Strictly mark the citation as excluded if it mentions review or pooled analysis which includes data from more than one study.
    5. If the study design is not clearly randomized controlled trials (RCTs) then exclude the study.
    """

    # Accumulate text from all pages
    for page_num in range(len(pdf_file)):
        page = pdf_file.load_page(page_num)
        text_chunk = page.get_text().lower()

        # Send chunk and receive decision
        decision += send_chunk_get_decision(text_chunk, context=decision)

    # Write the decision to a text file
    pdf_decision_file = pdf_file_path.replace(
        os.path.splitext(pdf_file_path)[1], "_decision.txt"
    )
    with open(pdf_decision_file, "w+") as file:
        file.write(decision)

    # Print the final decision
    print(decision)

except Exception as e:
    print(f"An error occurred: {str(e)}")


Apologies, but I can't provide the help you're looking for since there's no document or citation provided to review according to the inclusion and exclusion criteria.Apologies, but I can't provide the information you're looking for.Apologies, the citation to review seems to be missing. Could you please provide the citation that needs to be screened based on the provided inclusion and exclusion criteria?Apologies, as I can't provide the appropriate response without the citation or document that needs to be assessed. Could you please provide me with the document or citation?Apologies for the confusion, but I can't proceed since the citation isn't provided. Please provide the citation so I can assess and classify it.Apologies for any confusion but without a citation provided to screen against the inclusion and exclusion criteria, I am unable to classify the citation as 'Included' or 'Excluded'. Please provide the citation information and I will be glad to assist.Apologies, but I can't pro